In [50]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer #字典型特征值抽取模块
from sklearn.model_selection import GridSearchCV
import  pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder

# 训练集特征工程：

In [74]:
satis=pd.read_csv(r'E:\Users\Boeing\Desktop\kesci project\训练集.csv')
satis_data=satis.iloc[:,1:-1]#去除id
y_train=satis['satisfaction_level']#目标值

#更换列名
col = ['上次评分','项目数量','平均月工时','工龄','工伤','福利等级','五年内有升职','部门','工资水平','满意度']
name = {i:j for i,j in zip(satis_data.columns,col)}
data = satis_data.rename(columns=name)

#某些等级值个性化处理
data=data.replace({'medium':5,'low':9,'high':12,'a':12,'b':7,'e':12,'c':12,'d':6})

data_五年内=data.五年内有升职.replace({1:2,0:1})
data = pd.concat([data,data_五年内],axis = 1)
data=pd.concat([data.iloc[:,:6],data.iloc[:,7:]],axis = 1)
'''
x_train=data.replace({'IT':1,
                               'RandD':2,
                               'accounting':3,
                               'hr':4,
                               'management':5,
                               'marketing':6,
                               'product_mng':7,
                               'sales':8,
                               'support':9,
                               'technical':10})
'''
#部门独热编码
data1 = pd.get_dummies(data['部门'], columns=(data['部门'].values))
satis_data = pd.concat([data,data1],axis = 1)
x_train=satis_data.drop(columns=['部门'])
#'''

In [75]:
x_train

,上次评分,项目数量,平均月工时,工龄,工伤,福利等级,工资水平,五年内有升职,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical
0,0.99,3,161.39,2,0,12,5,1,0,0,1,0,0,0,0,0,0,0
1,1.00,5,226.22,6,0,7,9,1,0,0,0,0,0,1,0,0,0,0
2,0.91,5,199.45,2,0,12,5,1,0,0,0,0,0,0,0,1,0,0
3,0.51,3,235.14,3,0,12,9,1,0,0,0,0,0,0,0,1,0,0
4,0.89,3,219.91,2,0,12,9,1,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11994,0.89,4,150.45,3,0,12,12,1,0,0,1,0,0,0,0,0,0,0
11995,0.81,5,221.28,5,0,7,9,1,0,0,0,0,0,0,0,1,0,0
11996,0.60,4,136.36,4,1,12,5,1,0,0,0,0,0,0,0,1,0,0
11997,0.84,6,268.54,4,0,7,9,1,1,0,0,0,0,0,0,0,0,0


# 测试集数据特征工程：

In [76]:
test=pd.read_csv(r'E:\Users\Boeing\Desktop\kesci project\测试集.csv')
test_data=test.iloc[:,1:]#去除id
#更换列名
col = ['上次评分','项目数量','平均月工时','工龄','工伤','福利等级','五年内有升职','部门','工资水平']
name = {i:j for i,j in zip(test_data.columns,col)}
data = test_data.rename(columns=name)

#某些等级值个性化处理
data=data.replace({'medium':5,'low':9,'high':12,'a':12,'b':7,'e':12,'c':12,'d':6})

data_五年内=data.五年内有升职.replace({1:2,0:1})
data = pd.concat([data,data_五年内],axis = 1)
data=pd.concat([data.iloc[:,:6],data.iloc[:,7:]],axis = 1)
'''
x_test=data.replace({'IT':1,
                               'RandD':2,
                               'accounting':3,
                               'hr':4,
                               'management':5,
                               'marketing':6,
                               'product_mng':7,
                               'sales':8,
                               'support':9,
                               'technical':10})
'''
#部门独热编码
data1 = pd.get_dummies(data['部门'], columns=(data['部门'].values))
test_data = pd.concat([data,data1],axis = 1)
x_test=test_data.drop(columns=['部门'])
x_test
#'''

,上次评分,项目数量,平均月工时,工龄,工伤,福利等级,工资水平,五年内有升职,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical
0,0.78,4,260.71,2,0,12,5,1,0,0,0,0,0,0,0,0,0,1
1,0.96,7,275.38,5,0,7,9,1,0,0,0,0,0,1,0,0,0,0
2,0.57,4,207.29,5,0,12,9,1,0,0,0,0,0,0,0,1,0,0
3,0.89,5,150.70,4,0,12,5,1,0,0,0,0,0,0,0,0,0,1
4,0.93,3,141.23,2,0,12,5,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.45,3,132.90,2,0,12,9,1,0,0,0,0,0,0,0,0,0,1
2996,0.73,4,245.77,2,0,12,9,1,0,0,0,0,0,0,0,1,0,0
2997,0.75,4,242.90,4,0,12,9,1,0,0,0,0,0,0,1,0,0,0
2998,0.70,3,235.52,4,0,12,12,1,0,0,1,0,0,0,0,0,0,0


In [77]:
x_test

,上次评分,项目数量,平均月工时,工龄,工伤,福利等级,工资水平,五年内有升职,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical
0,0.78,4,260.71,2,0,12,5,1,0,0,0,0,0,0,0,0,0,1
1,0.96,7,275.38,5,0,7,9,1,0,0,0,0,0,1,0,0,0,0
2,0.57,4,207.29,5,0,12,9,1,0,0,0,0,0,0,0,1,0,0
3,0.89,5,150.70,4,0,12,5,1,0,0,0,0,0,0,0,0,0,1
4,0.93,3,141.23,2,0,12,5,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.45,3,132.90,2,0,12,9,1,0,0,0,0,0,0,0,0,0,1
2996,0.73,4,245.77,2,0,12,9,1,0,0,0,0,0,0,0,1,0,0
2997,0.75,4,242.90,4,0,12,9,1,0,0,0,0,0,0,1,0,0,0
2998,0.70,3,235.52,4,0,12,12,1,0,0,1,0,0,0,0,0,0,0


# 随机森林建模预测及网格搜索：

In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
param_grid=[
    {
        'n_estimators':[300,350,400,450],
        'max_features':['auto', 'sqrt', 'log2']
        
    }
]

#estimator = RandomForestRegressor(max_depth = None,min_samples_split = 2,n_jobs=-1)
estimator = RandomForestRegressor(n_jobs=-1)
rfr1 = GridSearchCV(estimator,param_grid=param_grid, cv=5 ,n_jobs=-1)
#rfr1 = RandomForestRegressor(n_estimators=100,n_jobs=-1)
rfr1.fit(x_train,y_train)
y_predict = rfr1.predict(x_test)
#rfr1.score(x_test,y_test)


In [56]:
    # 最佳参数：best_params_
    print("最佳参数：\n", rfr1.best_params_)
    # 最佳结果：best_score_
    print("最佳结果：\n", rfr1.best_score_)
    # 最佳估计器：best_estimator_
    print("最佳估计器:\n", rfr1.best_estimator_)

最佳参数：
 {'max_features': 'log2', 'n_estimators': 400}
最佳结果：
 0.5150124574726205
最佳估计器:
 RandomForestRegressor(max_features='log2', n_estimators=400, n_jobs=-1)


In [62]:
data1 = pd.DataFrame(y_predict)
data1.to_csv(r'C:\Users\Boeing\Desktop\kesci project\测试结果集0.5256.csv')

In [47]:
str(rfr1.best_estimator_)[22:-1]

"max_features='sqrt', n_estimators=115, n_jobs=-1"

In [48]:
str(rfr1.best_score_)

'0.5247769355998375'

# 随机森林网格搜索循环预测：

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import time
param_grid=[
    {
        'n_estimators':[300,350,450,500,550],
        'max_features':[0.5,0.6,0.7,0.4,'log2']
        
    }
]

estimator = RandomForestRegressor(max_depth = None,min_samples_split = 2,n_jobs=-1)

for i in range(6):
    rfr1 = GridSearchCV(estimator,param_grid=param_grid, cv=10 ,n_jobs=-1)
    #rfr1 = RandomForestRegressor(n_estimators=100,n_jobs=-1)
    rfr1.fit(x_train,y_train)
    y_predict = rfr1.predict(x_test)
    #rfr1.score(x_test,y_test)

    data1 = pd.DataFrame(y_predict)
    data1.to_csv(r'E:\Users\Boeing\Desktop\kesci project\循环测试7\%s,%s.csv'%(str(rfr1.best_score_),str(rfr1.best_estimator_)[22:-1]))
    if i%3==0:
        time.sleep(0)